# COGS 108 - EDA Checkpoint

# Names

- Nilou Shahbandi
- Leonardo Gonzalez
- Brian Chen
- Tin-Trung Pham
- Zhongkang(ken) Fang

<a id='research_question'></a>
# Research Question

*How do different pollutants in California impact the rate of cancer in individuals, and how does pollutants affect rate of Asthma?*

# Setup

In [31]:
## YOUR CODE HERE
import pandas as pd 
import numpy as np

aqi_2010 = pd.read_csv('data/aqireport2010.csv')
county_cancer = pd.read_csv('data/countyCancer.csv')

# Data Cleaning

Describe your data cleaning steps here.

In [7]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

In [32]:
# Data cleaning for AQI_2010
aqi_2010.drop(['# Days with AQI', 'Moderate', 'Unhealthy', 'Very Unhealthy', 'County Code',
                'Hazardous', 'AQI Maximum', '# Days CO', '# Days NO2', '# Days O3', '# Days PM2.5', '# Days PM10'  ], 
                 inplace = True, axis=1)
aqi_2010.columns = ['county', 'Good days', 'Unhealthy days', 'AQI 90th Percentile', 'AQI Median']
aqi_2010['county'] = aqi_2010['county'].apply(standardize_region)
aqi_2010.head()

,county,Good days,Unhealthy days,AQI 90th Percentile,AQI Median
0,alameda,245,4,68.0,43.0
1,amador,302,10,64.0,35.0
2,butte,208,13,84.0,47.0
3,calaveras,271,12,84.0,41.0
4,colusa,322,1,49.0,38.0


In [33]:
# Data Cleaning for county_cancer_2010
# filter year 2010
county_cancer_2010 = county_cancer[county_cancer['Year'] == 2010]
county_cancer_2010 = county_cancer_2010[county_cancer_2010['Age'] == 'All Ages']
county_cancer_2010.drop(['Sex', 'Race/Ethnicity', 'Age', 'Stage at Diagnosis', 'Age-adjusted Rate Standard Error', 'Year',
                         'Age-adjusted Rate Lower Confidence Interval', 'Age-adjusted Rate Upper Confidence Interval'], 
                 inplace = True, axis=1)
county_cancer_2010['California Region'] = county_cancer_2010['California Region'].apply(standardize_region)
county_cancer_2010.columns = ['county', 'rate per 100,000', 'count']
county_cancer_2010.head()

,county,"rate per 100,000",count
0,alameda,44.8118,646
1,butte,63.0616,171
2,contra costa,50.2429,536
3,el dorado,54.2364,124
4,fresno,51.7782,401


In [36]:
# combine aqi and cancer data
# year: 2010

# drop the county with combined cancer value
# we drop them instead of taking the average because there are significant differences for the AQI values 
county_cancer_2010 = county_cancer_2010.drop(county_cancer_2010.index[39:47])


aqi_cancer_2010 = pd.merge(county_cancer_2010, aqi_2010, on='county')
aqi_cancer_2010.head()

,county,"rate per 100,000",count,Good days,Unhealthy days,AQI 90th Percentile,AQI Median
0,alameda,44.8118,646,245,4,68.0,43.0
1,butte,63.0616,171,208,13,84.0,47.0
2,contra costa,50.2429,536,307,8,64.0,39.0
3,el dorado,54.2364,124,251,23,93.0,42.0
4,fresno,51.7782,401,99,65,130.0,70.0


In [30]:

def standardize_region(str_in):
    
    try:
        str_in = str_in.lower()
        str_in = str_in.strip()
        output = str_in

        if 'county, ca' in str_in:
            str_in = str_in.replace('county, ca', '')
            str_in = str_in.strip()
            output = str_in
            
        if 'county' in str_in:
            str_in = str_in.replace('county', '')
            str_in = str_in.strip()
            output = str_in
            
            
    except:
        output = np.nan
        
    return output

# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION